# 항만 체류 시간 예측 프로젝트

### 프로젝트 개요

항만은 글로벌 공급망과 국가 간 무역의 핵심 거점으로, 선박의 입항부터 출항까지의 체류 시간은 항만의 운영 효율성과 해운 기업의 정시 운항 능력에 직결되는 중요한 지표입니다.

특히 항만 내 선박 체류 시간은 터미널 혼잡도, 선석 배정의 효율성, 하역 인력 운영 등 다양한 물류 흐름과 운영 비용에 영향을 미치며, 해운사의 수익성과 고객 만족도에도 간접적인 영향을 줍니다.

이번 프로젝트에서는 해양수산부의 Port-MIS 입출항 데이터를 기반으로, 개별 선박의 입항 및 출항 정보, 선박 제원, 항만 및 기항지 정보를 활용하여 항만 내 선박 체류 시간을 예측하는 머신러닝 회귀 모델을 구축하고 성능을 평가합니다.


### 목표

- 선박 및 항만 관련 입출항 데이터를 기반으로, 항만 체류 시간(시간 단위)을 예측하는 회귀 모델을 개발

- 모델 성능은 MSE, R² 평가 지표를 통해 측정 및 비교

- 적재화물명, 적재톤수, 총톤수 등의 범주형/수치형 정보를 활용

- 체계적인 전처리(결측값 제거, 이상치 필터링, 인코딩 등) 및 파생 변수 생성 과정을 수행

- 모델 해석을 통해 체류 시간에 영향을 미치는 핵심 요인들을 분석

- 전처리 및 피처 엔지니어링을 통해 머신러닝 성능 향상을 위한 구조적 접근 진행


### 데이터 출처

해양수산부_해운항만물류정보_한중일물류 입출항정보_20240731.csv : [해양수산부 해운항만물류정보 한중일물류 입출항정보](https://www.data.go.kr/data/15133162/fileData.do) 


---

## 목차
1. 데이터 불러오기
2. 탐색적 데이터 분석 (EDA)
3. 데이터 전처리
4. 체류 시간 예측 회귀 모델
5. 모델 검증



## 1. 데이터 불러오기 

이번 프로젝트에서 필요한 라이브러리를 불러옵니다.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from tqdm import tqdm


# 깔끔한 출력을 위한 설정
import warnings

warnings.filterwarnings("ignore")  # 기술적 경고 숨기기

# 한글 폰트 설정
plt.rc("font", family="NanumGothic")
plt.rcParams["axes.unicode_minus"] = False

먼저 데이터를 확인해 볼까요?

In [ ]:
# 데이터 불러오기
df = pd.read_csv(
    "data/해양수산부_해운항만물류정보_한중일물류 입출항정보_20240731.csv",
    encoding="cp949",
)

# 데이터 크기 확인
print("데이터 shape:", df.shape)
print("\n 데이터의 상위 5행 : ")
print(df.head())

이 데이터에는 각 선박의 입항 및 출항 시각, 선박 제원, 입출항 항만 및 기항지 정보 등 다양한 항만 운영 및 선박 특성 관련 변수가 포함되어 있으며, 이러한 요소들은 항만 내 체류 시간에 영향을 미치는 핵심 요인으로 작용합니다. 이 변수들을 바탕으로 회귀 모델의 입력값을 구성하게 됩니다.

이 데이터에서 컬럼은 다음과 같습니다.

| 컬럼명         | 설명                               | 컬럼명         | 설명                               |
| ----------- | -------------------------------- | ----------- | -------------------------------- |
| **항구청코드**   | 입출항을 관리하는 관할 항만청의 고유 코드 (부산)     | **입출항구분**   | 해당 행이 입항인지 출항인지 구분 (입항, 출항)      |
| **호출부호**    | 선박의 무선 통신용 고유 식별 코드 (Call Sign)  | **입항년도**    | 선박이 입항한 연도                       |
| **입항횟수**    | 해당 연도에 선박이 입항한 누적 횟수             | **신고일시**    | 해당 입출항 신고가 이루어진 날짜 및 시각          |
| **계선시설명**   | 선박이 정박한 부두 또는 선석(계선시설)의 이름       | **적재화물명**   | 선박에 적재된 주요 화물의 이름 또는 종류          |
| **적재톤수**    | 선박에 실린 전체 화물의 무게 (톤 단위)          | **환적톤수**    | 다른 선박으로 환적 예정인 화물의 무게            |
| **양적하화물톤**  | 하역한 전체 화물의 무게 (톤 단위)             | **위험물톤수**   | 위험물로 분류된 화물의 무게 (톤 단위)           |
| **징수결정톤수**  | 항만 사용료나 하역료 등 부과 기준이 되는 톤수       | **총톤수**     | 선박의 총 내부 용적 (Gross Tonnage, G/T) |
| **내외항구분**   | 해당 선박이 내항(국내) 또는 외항(국제) 항로인지 구분  | **선박국적명**   | 선박이 등록된 국가명 (국적)                 |
| **선박종류명**   | 선박의 분류 (풀컨테이너선, 냉동.냉장선, 일반화물선 등) | **선박명**     | 선박의 이름                           |
| **입출항일시**   | 선박의 입항 또는 출항 시각 (입출항구분과 함께 해석)   | **입항목적명**   | 입항 목적 (하역, 승선, 연료 보급 등)          |
| **전출항지국가명** | 직전 출항지의 국가명                      | **전출항지항구명** | 직전 출항지의 항만 이름                    |
| **차항지국가명**  | 다음 기항 예정 국가명                     | **차항지항구명**  | 다음 기항 예정 항만 이름                   |
| **목적지국가명**  | 선박이 향하는 최종 목적지의 국가명              | **목적지항구명**  |                                  |

이렇게 이번 프로젝트에서 사용할 데이터들을 확인해보았습니다. 

이제부터는 탐색적 데이터 분석을 수행하며 데이터를 이해하고 어떻게 전처리할지 고민해 보도록 하겠습니다. 

## 2. 탐색적 데이터 분석 (EDA)

EDA는 모델 학습 전 데이터를 깊이 이해하는 과정으로, 변수 간의 관계, 분포, 결측값, 이상치 여부 등을 확인하는 데 필수적입니다.

우선 데이터의 데이터 타입과 분포를 확인해보겠습니다.

In [ ]:
# 데이터프레임 컬럼 확인
df.columns

In [ ]:
# 데이터 타입 및 결측치 확인
df.info()

이 데이터셋은 총 28개의 컬럼으로 구성되어 있으며, 문자형(**object**) 변수와 수치형 변수(**float64**, **int64**)가 혼합된 구조입니다.

특히 `입출항구분`, `선박종류명`, `내외항구분`, `항만청코드` 등은 명목형 범주 변수로, 머신러닝 모델에 투입하기 전 반드시 인코딩 처리가 필요합니다.

또한 `입출항일시`, `신고일시`, `등록일시`, `수정일시` 등 날짜와 시간 정보를 포함하는 컬럼들은 **datetime** 형식으로 변환하여 사용할 수 있습니다.

반면, `적재톤수`, `총톤수`, `징수결정톤수` 등은 정량적인 수치형 변수로 그대로 사용할 수 있습니다.

### 2.1. 입출항 트렌드 분석

`입항년도`와 `입출항일시`를 기준으로 연도별, 월별 입출항량 변화 추이를 분석하겠습니다. 특정 계절 또는 기간에 입출항이 집중되는지 여부를 파악하는 과정입니다.

먼저 `pd.to_datetime()` 함수를 사용해 문자열(**string**) 형태로 저장된 `입출항일시` 정보를 **datetime** 자료형으로 변환합니다. 이렇게 변환된 값은 연도별, 월별, 시간대별 분석, 시간 차 계산(`체류시간`) 등의 시계열 분석이 가능합니다.

In [ ]:
df["입출항일시"] = pd.to_datetime(df["입출항일시"], errors="coerce")

입출항 시점을 연-월 단위로 그룹화하기 위해 새로운 컬럼 **입항연월**을 생성합니다.

**datetime** 자료형으로 변환한 `입출항일시` 컬럼을 `.dt.to_period("M")`를 사용하여 연-월(YYYY-MM) 형태의 Period 형식으로 변환합니다. 이 과정을 통해 `2023-07-15 08:00:00`로 저장된 데이터를 `2023-07`로 변환할 수 있습니다.

In [ ]:
df["입항연월"] = df["입출항일시"].dt.to_period("M")

df["입항연월"]

`입항연월`과 `입출항구분` 조합별로 <b>건수(행 수)</b>를 집계하여 월별 입출항 트렌드를 분석합니다.

` .unstack()`로 `입출항구분`을 컬럼으로 펼쳐 `입항`과 `출항`을 각각의 열로 만듭니다.

In [ ]:
monthly_trend = df.groupby(["입항연월", "입출항구분"]).size().unstack(fill_value=0)

monthly_trend

`입항`과 `출항`의 월별 추이 변화를 한눈에 비교할 수 있는 선그래프를 그려봅시다.

In [ ]:
plt.figure(figsize=(15, 6))
monthly_trend.plot(kind="line", marker="o", figsize=(15, 6))
plt.title("월별 입출항 건수 추이")
plt.xlabel("연-월")
plt.ylabel("입출항 건수")
plt.xticks(rotation=45)

5월까지는 입항과 출항 모두 거의 0건에 가깝다가 6월부터 입출항 건수가 급격히 증가합니다. 이를 통해 본격적인 운항 시작 시점 또는 관측 시작 시점으로 추정된다고 판단할 수 있습니다.

월별로 분석해 보겠습니다.
- 6월: 입항은 약 2,300건, 출항은 약 1,900건입니다. 입항이 출항보다 더 많아 선박 체류 증가 가능성을 나타냅니다.

- 7월: 입항은 약 3,900건, 출항은 약 3,000건입니다. 전월 대비 입출항 모두 증가했고 특히 입항은 큰 폭의 증가했습니다. 여름 성수기나 항만 이용 증가 시기로 판단됩니다.

### 2.2. 화물 톤수 및 화물 종류별 분석

각 화물 톤수 변수의 전체 분포 형태를 파악하고, 이상치 유무, 중심 경향 등을 시각적으로 확인해 보겠습니다.

seaborn 라이브러리의 `histoplot()`을 사용해 각 톤수 변수의 히스토그램과 커널 밀도 추정(KDE) 곡선을 함께 시각화합니다.


In [ ]:
fig1, axes1 = plt.subplots(1, 2, figsize=(15, 5))
sns.histplot(df["적재톤수"], bins=50, kde=True, ax=axes1[0]).set_title("적재톤수 분포")
sns.histplot(df["양적하화물톤"], bins=50, kde=True, ax=axes1[1]).set_title("양적하화물톤 분포")
plt.tight_layout()


현재 시각화된 히스토그램을 보면 대부분의 데이터가 왼쪽에 밀집되어 있고, x축이 긴 꼬리를 가진 비대칭 형태를 보이고 있습니다. 이는 일부 극단적인 값(이상치) 때문에 전체 분포가 왜곡되어 발생하는 현상입니다.

따라서, 사분위수 기반의 기준(IQR)을 활용해 이상치를 제거하고, 보다 대표적인 데이터 분포를 확인해 보겠습니다.

In [ ]:
def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]


# 이상치 제거 대상 컬럼
target_columns = ["적재톤수", "양적하화물톤"]

# 각 컬럼별 이상치 제거 적용
for col in target_columns:
    df = remove_outliers_iqr(df, col)

# 결과 확인
print(f"이상치 제거 후 데이터 크기: {df.shape}")

이제 다시 데이터를 확인해 보겠습니다.

In [ ]:
fig1, axes1 = plt.subplots(1, 2, figsize=(15, 5))
sns.histplot(df["적재톤수"], bins=50, kde=True, ax=axes1[0]).set_title("적재톤수 분포")
sns.histplot(df["양적하화물톤"], bins=50, kde=True, ax=axes1[1]).set_title("양적하화물톤 분포")
plt.tight_layout()

이전 그래프에서는 이상치로 인해 x축이 넓게 분포하면서, 대부분의 데이터가 0 근처에 몰려 있는 것처럼 보였습니다. 이로 인해 실제 주요 분포 구간의 패턴을 시각적으로 파악하기 어려웠습니다.

반면, 이상치를 제거한 이후의 그래프에서는 데이터가 집중된 구간이 더 명확히 드러나며, 전체적인 분포의 형태를 보다 직관적으로 확인할 수 있습니다. 특히 `적재톤수`와 `양적하화물톤` 모두에서, 소수의 극단값에 의해 왜곡되었던 밀도 분포가 완화되어 실제 데이터의 중심 경향과 분포의 꼬리 부분이 더욱 뚜렷하게 나타납니다.

다음으로 화물 종류별 평균 적재톤수 상위 20개 항목을 막대그래프로 시각화해 보겠습니다.

`적재화물명`을 기준으로 그룹화하고 각 그룹에 대해 `적재톤수`의 평균값을 계산합니다. 계산한 평균값을 내림차순으로 정렬하고 그중 상위 20개 항목만 추출하여 `top_cargo`에 저장합니다. 즉, `top_cargo`는 평균적으로 가장 무거운 화물을 상위 20개까지 뽑아낸 것입니다.

In [ ]:
# 화물 종류별 평균 적재톤수 (상위 20개)
top_cargo = df.groupby("적재화물명")["적재톤수"].mean().sort_values(ascending=False).head(20)

top_cargo

이 `top_cargo`를 사용하여 막대그래프를 그려보겠습니다.

In [ ]:
plt.figure(figsize=(12, 6))
sns.barplot(x=top_cargo.values, y=top_cargo.index)
plt.title("화물 종류별 평균 적재톤수 (상위 20개)")
plt.xlabel("평균 적재톤수")
plt.ylabel("화물 종류")

아연은 평균 적재톤수가 1만 톤 이상으로 가장 높습니다. 이는 야연이 대형 선박으로 주로 운송되고 있음을 의미합니다. 다음으로 동, 알루미늄, 펄프는 각각 6,000~7,000톤 수준으로 중량 화물로 분류됩니다. 비철금속 및 원자재 성격이 강해 대규모 단위 운송이 이뤄지는 것으로 보입니다.

각종비금속제품, 연, 기타비금속 등은 대부분 원자재 또는 중간재로서 대량 운송이 필요한 품목입니다. 돌·시멘트·석면제품, 유리, 도석·소금 등 건설·건축 자재도 눈에 띄며, 이들 역시 단위당 무게가 크기 때문에 평균 적재량이 높게 나옵니다.

비료, 차량, 곡물, 연료·에너지, 광,슬랙,회 등은 중간 이하에 분포되어 있으며, 상대적으로 적재량 단위가 작거나 분산 수송되는 특성이 있습니다.

## 3. 데이터 전처리

데이터 전처리는 본격적인 분석이나 모델링에 앞서 데이터의 품질을 개선하고, 의미 있는 정보를 추출 가능한 형태로 변환하는 필수 작업입니다. 원본 데이터는 종종 불완전하거나, 중복되거나, 분석 목적과 맞지 않는 구조를 가지는 경우가 많기 때문에, 이를 정제하지 않고 분석하면 오류, 편향, 비효율적인 결과가 발생할 수 있습니다.

이번 선박 입출항 데이터에서도 다양한 변수들이 존재하며, 이를 효과적으로 활용하기 위해 전처리 작업을 순차적으로 수행합니다.

### 3.1. 중복값, 이상치, 결측치 제거

**중복값**, **이상치**, **결측치** 제거는 데이터 전처리에서 매우 중요한 단계입니다. 이 세 가지 처리 과정은 데이터의 신뢰성과 예측 성능 향상을 위해 반드시 수행되어야 합니다.

#### 중복값 제거

동일한 선박이 동일한 시각에 동일한 정보로 두 번 이상 기록되는 경우 등 중복된 행이 존재할 수 있습니다. 중복값은 데이터의 왜곡을 일으키고, 모델 학습 시 편향된 결과를 초래할 수 있습니다.

전체 컬럼을 기준으로 중복값의 개수를 확인하고 제거하겠습니다.

In [ ]:
print(df.duplicated().sum())

In [ ]:
df = df.drop_duplicates()

print(f"중복값 제거 후 데이터 크기: {df.shape}")

11,662개였던 데이터가 11,283개로 줄어든 것을 확인할 수 있습니다.


#### 이상치 제거

이상치는 데이터 분포에서 극단적으로 벗어난 값으로, 대부분 입력 오류, 측정 오류 또는 비정상적 상황에서 발생합니다. 이상치는 평균, 분산, 회귀 계수 등에 큰 영향을 주어 모델의 왜곡을 초래하고, 예측 성능을 저하시킵니다.

이전에 `적재톤수`, `환적톤수`는 이상치 제거 완료했기 때문에 `총톤수`에 대해서 이상치를 제거하겠습니다. (`양적하화물톤`과 `위험물톤수`, `징수결정톤수`는 크게 데이터분포값이 차이나지 않습니다.)

In [ ]:
# 이상치 제거 대상 컬럼
target_columns = ["총톤수"]

# 각 컬럼별 이상치 제거 적용
for col in target_columns:
    df = remove_outliers_iqr(df, col)

# 결과 확인
print(f"이상치 제거 후 데이터 크기: {df.shape}")

11,283개였던 데이터가 9,771개로 줄어든 것을 확인할 수 있습니다.

#### 결측치 제거

결측치를 제거하기 전, 불필요한 컬럼을 제거하겠습니다. 삭제할 컬럼명은 다음과 같습니다.

- 불필요한 정보 : `신고일시`, `등록일시`, `수정일시`, `항구청코드`, `입항횟수`, `입항목적명`, `선박국적명`, `선박종류명`
- 출항/목적지 관련 세부 위치 정보: `전출항지국가명`, `전출항지항구명`, `차항지국가명`,`목적지국가명`, `목적지항구명`, `계선시설명`
- 중간 파생 변수: `입항연월`

In [ ]:
drop_columns = [
    "신고일시", "등록일시", "수정일시", "입항연월",
    "전출항지국가명", "전출항지항구명",
    "차항지국가명", "차항지항구명",
    "목적지국가명", "목적지항구명",
    "항구청코드", "입항횟수",
    "계선시설명", "입항목적명",
    "선박국적명", "선박종류명"
]

df = df.drop(columns=drop_columns)

df

체류시간 예측에 필요한 컬럼 13개를 제외하고 나머지 컬럼은 모두 제거했습니다.

다음으로는 결측치를 제거하겠습니다. 결측치가 존재하는 컬럼을 확인합니다.

In [ ]:
df.isnull().sum()

현재 데이터프레임 `df`에는 결측치가 존재하지 않습니다.

### 3.2. 체류시간 컬럼 생성

선박이 항구에 머문 시간을 파악하여 물류 흐름이나 항만 운영 효율성을 분석하기 위해 `호출부호`, `선박명`, `입출항구분`, `입출항일시` 컬럼을 사용하여 체류시간 컬럼을 생성하겠습니다.

입출항이 여러 번 있는 선박은 구분이 필요하므로 `호출부호`와 `선박명` 컬럼을 조합한 "호출부호_선박명" 형태의 `선박ID` 컬럼을 생성합니다.

In [ ]:
# 선박 식별을 위한 기준 컬럼: 호출부호 + 선박명 조합 (호출부호만으로도 가능)
df["선박ID"] = df["호출부호"].fillna("UNKNOWN") + "_" + df["선박명"].fillna("UNKNOWN")

df["선박ID"]

입항과 출항을 구분하여 두 개의 DataFrame으로 분리합니다.

각 DataFrame에는 선박ID와 해당 시각만 포함되며, 컬럼명을 각각 `입항일시`, `출항일시`로 변경합니다.

In [ ]:
# 입항 / 출항 각각 분리
arr_df = df[df["입출항구분"] == "입항"][["선박ID", "입출항일시"]].rename(columns={"입출항일시": "입항일시"})
dep_df = df[df["입출항구분"] == "출항"][["선박ID", "입출항일시"]].rename(columns={"입출항일시": "출항일시"})

입항 시각과 가장 가까운 출항 시각을 1:1로 매칭하기 위해 `matched_records` 리스트를 생성하겠습니다.

`선박ID`는 각 선박을 고유하게 식별할 수 있는 값입니다. `groupby`를 통해 각 선박에 대해 입항 데이터들과 출항 데이터들을 각각 따로 묶을 수 있습니다. 이후에 같은 `선박ID`를 가진 입항-출항 데이터를 매칭할 수 있게 됩니다.

In [ ]:
# 결과 저장용 리스트
matched_records = []

# 선박ID 기준으로 입항-출항 그룹핑
grouped_arr = arr_df.groupby("선박ID")
grouped_dep = dep_df.groupby("선박ID")

아래 `for`문을 통해 각 선박(`ship_id`)에 대한 입항 기록 그룹(`arr_group`)을 반복하면서 처리합니다.

출항 데이터가 없으면 체류시간을 계산할 수 없기 때문에 `선박ID`에 해당하는 출항 기록이 없는 경우는 건너뜁니다.

해당 선박의 출항 시각들만 모은 시리즈(`dep_group`)를 얻고, 시각순으로 정렬하여 `dep_times`에 저장합니다.

그 다음 이 선박에 대해 모든 입항 시각(`arr_time`)을 하나씩 순회합니다.

현재 입항 시간(`arr_time`)을 가져오고 출항 시간(`dep_times`) 중에서 입항 이후에 일어난 출항만 필터링해 `valid_dep`에 저장합니다. 즉, `valid_dep`는 해당 입항에 대응할 수 있는 유효한 출항 후보들입니다.

입항 이후 출항이 하나라도 존재하면, 그 중 가장 이른 출항 시간(`iloc[0]`)을 `closest_dep`에 저장합니다.

이렇게 구한 `closest_dep`와 매칭되는 `arr_time`의 시간 차, 즉 체류 시간(`dwell_time`)에 저장합니다. 이때 저장되는 값은 시간 단위이기 때문에 **3600**으로 나눠줍니다.

이론적으로 출항은 입항보다 늦어야 하므로, 음수 체류시간은 데이터 오류입니다. 체류시간이 양수일 경우 정상적인 입항-출항 페어링으로 판단하고 하나의 **dictionary** 형태로 결과를 리스트에 저장합니다.

In [ ]:
for ship_id, arr_group in tqdm(grouped_arr):
    if ship_id not in grouped_dep.groups:
        continue

    dep_group = grouped_dep.get_group(ship_id)
    dep_times = dep_group["출항일시"].sort_values().reset_index(drop=True)

    for _, arr_row in arr_group.iterrows():
        arr_time = arr_row["입항일시"]
        # arr_time 이후의 출항 시각 중 가장 빠른 것
        valid_dep = dep_times[dep_times > arr_time]
        if not valid_dep.empty:
            closest_dep = valid_dep.iloc[0]
            dwell_time = (closest_dep - arr_time).total_seconds() / 3600
            if dwell_time > 0:
                matched_records.append(
                    {
                        "선박ID": ship_id,
                        "입항일시": arr_time,
                        "출항일시": closest_dep,
                        "체류시간": dwell_time,
                    }
                )

결과를 DataFrame 형태로 확인해 봅시다.

In [ ]:
matched_df = pd.DataFrame(matched_records)

matched_df

기존 데이터프레임 `df`에 `체류시간` 컬럼을 추가합니다. 또한 병합과정에서 생성될 수 있는 중복 컬럼을 삭제합니다.

In [ ]:
df_arr = df[df["입출항구분"] == "입항"].copy()
df_arr = df_arr.rename(columns={"입출항일시": "입항일시"})

In [ ]:
df = pd.merge(matched_df, df_arr, on=["선박ID", "입항일시"], how="inner")
df = df.drop_duplicates()

df

이제 의미가 중복된 컬럼과 중간 파생 컬럼을 삭제하겠습니다.

In [ ]:
drop_columns = ["선박ID", "입출항구분", "호출부호", "선박명", "입항일시", "출항일시"]
df = df.drop(columns=drop_columns)

df

또한 현재 데이터프레임 df에서 유니크값이 1개인 컬럼을 찾아 해당 컬럼을 삭제하겠습니다.

In [ ]:
df.nunique(dropna=True)

`입항년도`, `내외항구분` 컬럼의 값이 1개인 것을 확인했으니 해당 두 컬럼을 삭제하겠습니다.

In [ ]:
drop_columns = ["입항년도", "내외항구분"]

df = df.drop(columns=drop_columns)

df

### 3.3 범주형 변수 인코딩

범주형(categorical) 변수는 일반적으로 문자열로 표현되며, 머신러닝 알고리즘은 이러한 비수치형 데이터를 직접 처리할 수 없습니다. 따라서 모델링 전에 반드시 수치형으로 변환해줘야 합니다. 이 과정을 <b>인코딩(encoding)</b>이라고 하며, 각 변수의 특성과 범주의 수에 따라 적절한 인코딩 방식을 선택해야 합니다.

현재 데이터셋에서 `적재화물명`이 범주형 변수로 분류됩니다.

In [ ]:
df.nunique(dropna=True)

위 코드는 각 컬럼별 고윳값 갯수를 출력한 것입니다. `적재화물명` 컬럼의 범주는 순서가 없기 때문에 원-핫 인코딩(One-Hot Encoding)이 적절하나, 범주의 수가 35개로 너무 많기 때문에 그대로 인코딩 시 **차원이 급격히 증가**하게 됩니다. 따라서 상위 빈도 10개만 선택하고 나머지는 "기타"로 분류하는 작업이 필요합니다.


In [ ]:
# 상위 10개 화물 종류 추출
top_cargos = df["적재화물명"].value_counts().nlargest(10).index

print(top_cargos)

In [ ]:
# 나머지는 '기타'로 처리
df["적재화물명"] = df["적재화물명"].apply(
    lambda x: x if x in top_cargos else "기타"
)

df.nunique()

이제 `적재화물명`컬럼에 대해 원-핫 인코딩을 수행하겠습니다.

범주를 개별 컬럼으로 나누고, 해당 범주에 해당하면 `True`, 아니면 `False`으로 표시하겠습니다. 생성되는 컬럼 이름 앞에 "화물_"이라는 접두어를 붙입니다. (`화물_어류`, `화물_유기화합물` 등) 이때, `drop_first=True`로 설정합니다. 

원본 데이터프레임에서 이제는 필요가 없어진 `적재화물명` 컬럼을 삭제하고 앞 단계에서 생성된 원-핫 인코딩 결과 (`encoded_df`)와 원본 데이터프레임을 합칩니다.

In [ ]:
# One-hot 인코딩
encoded_df = pd.get_dummies(df["적재화물명"], prefix="화물", drop_first=False)

df = df.drop(columns=["적재화물명"])
df = pd.concat([df, encoded_df], axis=1)

df

### 3.4. 수치형 변수 변환

수치형 변수들 중 `총톤수`나 `적재톤수`와 같이 값의 분포가 <b>심하게 왜곡(왜도)</b>되어 있거나, 범위 차이가 큰 경우, 그대로 사용할 경우 모델 학습 시 특정 변수의 영향력이 과도하게 커지거나 변수 간 비교가 어려울 수 있습니다. 따라서 이 문제를 완화하기 위해 <b>로그 변환(log transformation)</b>과 <b>스케일링(scaling)</b>을 수행합니다.

#### 로그 변환

큰 값을 눌러주고 작은 값은 거의 변화시키지 않으며 분포의 왜도를 완화해 정규성에 가까운 형태로 변환하기 위해 로그 변환 과정을 수행합니다.

로그 변환 전 수치형 변수의 분포를 히스토그램과 KDE로 살펴보겠습니다.

In [ ]:
fig1, axes1 = plt.subplots(4, 2, figsize=(15, 10))
sns.histplot(df["적재톤수"], bins=50, kde=True, ax=axes1[0, 0]).set_title("적재톤수 분포 (로그 변환 전)")
sns.histplot(df["환적톤수"], bins=50, kde=True, ax=axes1[0, 1]).set_title("환적톤수 분포 (로그 변환 전)")
sns.histplot(df["양적하화물톤"], bins=50, kde=True, ax=axes1[1, 0]).set_title("양적하화물톤 분포 (로그 변환 전)")
sns.histplot(df["위험물톤수"], bins=50, kde=True, ax=axes1[1, 1]).set_title("위험물톤수 분포 (로그 변환 전)")
sns.histplot(df["징수결정톤수"], bins=50, kde=True, ax=axes1[2, 0]).set_title("징수결정톤수 분포 (로그 변환 전)")
sns.histplot(df["총톤수"], bins=50, kde=True, ax=axes1[2, 1]).set_title("총톤수 분포 (로그 변환 전)")
sns.histplot(df["체류시간"], bins=50, kde=True, ax=axes1[3, 0]).set_title("체류시간 분포 (로그 변환 전)")
plt.tight_layout()

이제 수치형 변수들에 대해 로그 변환을 수행하겠습니다. 로그 변환은 0 이하의 값이 있으면 적용이 불가능하므로 보통 `np.log1p()`를 써서 `log(1 + x)` 형태로 처리합니다.

In [ ]:
numeric_cols = ["적재톤수", "환적톤수", "양적하화물톤", "위험물톤수", "징수결정톤수", "총톤수", "체류시간"]

for col in numeric_cols:
    df[col] = np.log1p(df[col])

변환된 수치형 변수의 분포를 히스토그램과 KDE로 살펴보겠습니다.

In [ ]:
fig1, axes1 = plt.subplots(4, 2, figsize=(15, 10))
sns.histplot(df["적재톤수"], bins=50, kde=True, ax=axes1[0, 0]).set_title("적재톤수 분포 (로그 변환 후)")
sns.histplot(df["환적톤수"], bins=50, kde=True, ax=axes1[0, 1]).set_title("환적톤수 분포 (로그 변환 후)")
sns.histplot(df["양적하화물톤"], bins=50, kde=True, ax=axes1[1, 0]).set_title("양적하화물톤 분포 (로그 변환 후)")
sns.histplot(df["위험물톤수"], bins=50, kde=True, ax=axes1[1, 1]).set_title("위험물톤수 분포 (로그 변환 후)")
sns.histplot(df["징수결정톤수"], bins=50, kde=True, ax=axes1[2, 0]).set_title("징수결정톤수 분포 (로그 변환 후)")
sns.histplot(df["총톤수"], bins=50, kde=True, ax=axes1[2, 1]).set_title("총톤수 분포 (로그 변환 후)")
sns.histplot(df["체류시간"], bins=50, kde=True, ax=axes1[3, 0]).set_title("체류시간 분포 (로그 변환 후)")
plt.tight_layout()

기존 히스토그램과 비교했을 때 보다 정규분포에 가까운 형태로 바뀐 것을 볼 수 있습니다.

#### 스케일링

여러 변수 간 값의 <b>스케일(범위)</b>이 다르면 거리 기반 모델(KNN, SVM, 딥러닝 등) 또는 정규화가 필요한 알고리즘에서 문제 발생 가능성이 있습니다. 따라서 `StandardScaler`를 사용하여 수치형 변수를 표준화하도록 하겠습니다.

In [ ]:
# 스케일링
scaler = StandardScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

## 4. 체류 시간 예측 회귀 모델

모델링 단계에서는 화물 정보를 바탕으로 체류시간을 예측하는 회귀 모델을 구현해 보겠습니다. 사용하는 회귀 알고리즘은 랜덤 포레스트(`RandomForestRegressor`)입니다.

### 4.1. 데이터 분할

입력 변수(`X`)와 출력 변수(`y`)를 정의하고 학습용과 테스트용 데이터를 분할하겠습니다.

- 입력 변수 `X` : `체류시간` 컬럼을 제외한 나머지 컬럼
- 출력 변수 `y` : `체류시간` 컬럼

In [ ]:
X = df.drop(columns=["체류시간"])
y = df["체류시간"]

`train_test_split()` 메서드를 사용하여 학습용, 테스트용 데이터를 분할하겠습니다.

이때 전체 데이터의 20%를 테스트용, 80%를 학습용으로 사용하고 랜덤 분할의 결과가 항상 동일하도록 시드를 **42**로 고정합니다.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("학습용 데이터 갯수 : ", len(X_train))
print("테스트용 데이터 갯수 : ", len(X_test))

### 4.2. 모델 학습

랜덤 포레스트 회귀 모델(`RandomForestRegressor`)을 사용하여 체류 시간 예측을 수행합니다. 이 모델은 여러 개의 결정 트리를 앙상블하여 예측 성능을 높이는 모델입니다.

100개의 결정 트리를 사용합니다. 결정 트리의 갯수는 클수록 안정적이지만 느려집니다. 또한 각 트리의 최대 깊이를 10으로 제한하여 과적합을 방지합니다.

In [ ]:
model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)

model.fit(X_train, y_train)

## 5. 모델 검증

학습된 모델을 사용해 <b>테스트 데이터(X_test)</b>에 대한 체류시간을 예측하고 평가 지표를 통해 모델의 성능을 확인합니다.

테스트 데이터 `X_test`를 사용해 체류 시간을 예측합니다. 예측 결과는 `y_pred`에 저장됩니다.

In [ ]:
y_pred = model.predict(X_test)

### 5.1. MSE (Mean Squared Error)

**MSE**는 평균 제곱 오차 (Mean Squared Error)의 약자로, 머신 러닝에서 회귀 모델의 성능을 평가하는 지표로 사용됩니다. 예측값과 실제값 사이의 차이를 제곱하여 평균낸 값으로, **값이 작을수록** 모델의 예측 정확도가 높음을 의미합니다. 

In [ ]:
mse = mean_squared_error(y_test, y_pred)

print("MSE : ", mse)

### R² Score

**R² Score**는 결정계수를 의미하며 모델이 데이터의 분산을 얼마나 설명하는지 나타내는 지표입니다. 1에 가까울수록 우수한 성능을 나타내며 0이면 평균값 수준의 예측입니다. 음수일 경우 모델이 무작위 예측보다 못하다는 것을 의미합니다.

In [ ]:
r2 = r2_score(y_test, y_pred)

print("R² Score:", r2)

지금까지 진행한 데이터 분석, 전처리, 그리고 모델링 과정을 기반으로, 여러분만의 아이디어를 적용해 모델의 성능을 더욱 향상시켜 보세요. 예를 들어, 이상치 제거 기준을 바꾸거나, 새로운 파생변수를 생성하거나, 더 적합한 스케일링 기법을 선택하는 것도 좋은 방법입니다. 또한, 모델의 하이퍼파라미터를 조정하거나, 다른 머신러닝 알고리즘(XGBoost, Gradient Boosting 등)을 시도해 보는 것도 권장합니다.

여기서 중요한 점은 단순히 코드를 바꾸는 것이 아니라, **왜 그 방법이 더 효과적일 수 있는지** 논리적인 근거를 가지고 실험해보는 것입니다. 다양한 시도를 통해 결과를 비교하고, 성능이 얼마나 개선되었는지를 수치로 확인해 보세요. 이를 통해 여러분은 실제 데이터 기반 문제 해결에서 탐색적 사고와 실험적 접근의 중요성을 경험할 수 있습니다.

결국, 주어진 데이터를 어떻게 해석하고 다루느냐에 따라 예측 모델의 성능은 크게 달라질 수 있습니다. 창의적이고 분석적인 시도로 여러분만의 최적 모델을 찾아보시기 바랍니다.